paraphrasing the input first 

In [ ]:
from parrot import Parrot
import torch

phrases=[]
para=[]

with open("/content/drive/MyDrive/Colab Notebooks/output3.txt", "r") as f:
    for line in f :
      if line.strip()!="":
        phrases.append(line.strip())

parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")
for phrase in phrases:
  para_phrases = parrot.augment(input_phrase=phrase,
                               use_gpu=True,
                               do_diverse=True, 
                               max_length=100, 
                               adequacy_threshold = 0.6, 
                              fluency_threshold = 0.8)
  para.append(para_phrases)

lines=[]
with open("/content/drive/MyDrive/Colab Notebooks/output4.txt", "w") as file:
  for para_phrases in para:
    for para_phrase in para_phrases:
      new_sentence=''
      for char in para_phrase[0]:
        new_sentence += char
        if char =="\n":
          break
      lines.append(new_sentence)
      break
  file.writelines("\n".join(lines))

Text generation

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

with open("/content/drive/MyDrive/Colab Notebooks/output4.txt", "r") as f:
    input_sentences = [line.strip() for line in f]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


output_sentences = []
j=0
for j in range(0,len(input_sentences)):
    if input_sentences[j].strip() == "":
        continue  # Skip empty input sentences

    input_ids = tokenizer.encode(input_sentences[j], return_tensors="tf")
    # Generate new sentences
    sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    min_length=25,
    top_k=50, 
    top_p=0.95, 
    )

    for i, sample_output in enumerate(sample_outputs):
        generated_sentence = tokenizer.decode(sample_output, skip_special_tokens=True)
        new_sentence = ""
        for char in generated_sentence:
            new_sentence += char
            if char == "\n":
              break         
        output_sentences.append((phrases[j], new_sentence))

df = pd.DataFrame(output_sentences, columns=['original_sentence', 'generated_sentence'])

df.to_excel("/content/drive/MyDrive/Colab Notebooks/output4.xlsx", index=False)
print(output_sentences)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


[('This puts off some viewers.', "this can demotivate some people, but not all. A lot of people don't believe that God actually wants you to come here. He's saying that if he were to make you come out of his mouth, then you would have your"), ('A long adventure begins.', 'there begins a long journey toward their very end, as the book continues.\n'), ('Then begins a period of prosperity.', "a period of prosperity follows its transition into recession, with recession then taking on the character of a boom and a bust. At some point, you have to adjust to this. In today's world where growth is constant, you must adjust.\n"), ('The description of the public part stops here.', 'the description of the public part is stopped.\n'), ('It is therefore urgent to renew their outlets.', "in this context it is urgent to renew their outlets of information to prevent the proliferation of 'fake news' and false stories.\n"), ('The Treasury will remain independent of the Committee.', 'the department of tr

transaltion back to french

In [ ]:
import pandas as pd
from mtranslate import translate
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/output4.xlsx')

df['original_sentence_fr'] = df['original_sentence'].apply(lambda x: translate(x,"fr","en"))
df['generated_sentence_fr'] = df['generated_sentence'].apply(lambda x: translate(x,"fr","en"))

df.to_excel('/content/drive/MyDrive/Colab Notebooks/output4_translated.xlsx', index=False)
